## Example of the List `Monad` in use

In [1]:
twiceWhenEven :: [Integer] -> [Integer]
twiceWhenEven xs = do
    x <- xs
    if even x
        then [x*x, x*x]
        else [x*x]
        
twiceWhenEven [1..3]

--

twiceWhenEven' :: [Integer] -> [Integer]
twiceWhenEven' xs = do
    x <- xs
    if even x
        then [x*x, x*x]
        else []
        
twiceWhenEven' [1..3]

[1,4,4,9]

[4,4]

## Exercise: Either Monad

Implement the `Either` `Monad`.

In [2]:
data Sum a b =
    First a
    | Second b
    deriving (Eq, Show)

instance Functor (Sum a) where
    fmap f (First x)  = First x
    fmap f (Second y) = Second (f y)

instance Applicative (Sum a) where
    pure x = Second x
    
    (<*>) _ (First x)           = First x
    (<*>) (First x) _           = First x
    (<*>) (Second f) (Second y) = Second (f y)

instance Monad (Sum a) where
    return = pure
    
    (>>=) (First x) _  = First x
    (>>=) (Second y) f = f y

## Chapter Exercises

Write `Monad` instances for the following types. Use the `QuickCheck` properties we showed you to validate your instances.

In [3]:
import Test.QuickCheck
import Test.QuickCheck.Checkers
import Test.QuickCheck.Classes

-- 1.

data Nope a =
    NopeDotJpg
    deriving (Eq, Show)
    
instance Functor Nope where
    fmap _ NopeDotJpg = NopeDotJpg
    
instance Applicative Nope where
    pure _ = NopeDotJpg
    
    NopeDotJpg <*> NopeDotJpg = NopeDotJpg
    
instance Monad Nope where
    NopeDotJpg >>= _ = NopeDotJpg

In [27]:
-- 2.

import Prelude hiding (Left, Right)

data PhhhbbtttEither b a =
    Left a
    | Right b
    deriving (Eq, Show)
    
instance Functor (PhhhbbtttEither b) where
    fmap f (Left a)   = Left (f a)
    fmap _ (Right b)  = Right b

instance Monoid b => Applicative (PhhhbbtttEither b) where
  pure = Left
  
  Left f <*> Left a    = Left (f a)
  Right b <*> Right b' = Right (b `mappend` b')
  Right b <*> _        = Right b
  _ <*> Right b        = Right b
    
instance (Arbitrary a, Arbitrary b) => Arbitrary (PhhhbbtttEither a b) where
  arbitrary = do
    a <- arbitrary
    b <- arbitrary
    elements [Left a, Right b]
    
instance (Eq a, Eq b) => EqProp (PhhhbbtttEither b a) where
  (=-=) = eq

--

let trigger = undefined :: PhhhbbtttEither String (String, String, String)
quickBatch $ functor trigger
quickBatch $ applicative trigger


functor:
  identity: +++ OK, passed 500 tests.
  compose:  +++ OK, passed 500 tests.


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.

In [28]:
-- 3.

newtype Identity a =
    Identity a
    deriving (Eq, Ord, Show)

instance Functor Identity where
    fmap f (Identity a) = Identity (f a)

instance Applicative Identity where
    pure = Identity
    
    Identity f <*> Identity x = Identity (f x)

instance Monad Identity where
    return = pure
    
    Identity x >>= f = f x
    
instance Arbitrary a => Arbitrary (Identity a) where
  arbitrary = do
    a <- arbitrary
    return $ Identity a
    
instance (Eq a) => EqProp (Identity a) where
  (=-=) = eq
    
--

let trigger = undefined :: Identity (String, Int, String)
quickBatch $ functor trigger
quickBatch $ applicative trigger
quickBatch $ monad trigger


functor:
  identity: +++ OK, passed 500 tests.
  compose:  +++ OK, passed 500 tests.


applicative:
  identity:     +++ OK, passed 500 tests.
  composition:  +++ OK, passed 500 tests.
  homomorphism: +++ OK, passed 500 tests.
  interchange:  +++ OK, passed 500 tests.
  functor:      +++ OK, passed 500 tests.


monad laws:
  left  identity: +++ OK, passed 500 tests.
  right identity: +++ OK, passed 500 tests.
  associativity:  +++ OK, passed 500 tests.

Write the following functions using the methods provided by `Monad` and `Functor`. Using stuﬀ like identity and composition is fine, but it has to typecheck with types provided.

In [50]:
-- 1.

j :: Monad m => m (m a) -> m a
j = (>>= id)

j [[1, 2], [], [3]]

-- 2.

l1 :: Monad m => (a -> b) -> m a -> m b
l1 = fmap

--3.

l2 :: Monad m => (a -> b -> c) -> m a -> m b -> m c
l2 = liftM2

-- 4.

a :: Monad m => m a -> m (a -> b) -> m b
a ma mf = mf <*> ma

-- 5.

meh :: Monad m => [a] -> (a -> m b) -> m [b]
meh [] _     = return []
meh (x:xs) f = liftM2 (:) (f x) (meh xs f)

--6.

flipType :: Monad m => [m a] -> m [a]
flipType xs = meh xs id

[1,2,3]